### Financials Indicators

Import

In [2]:
import pandas_datareader as pdr
import pandas_datareader.data as web
import pandas as pd
from datetime import datetime
import numpy as np

import pymssql 

data to conect

In [3]:
_server=''
_user=''
_pass=''
_db=''

In [4]:
#Query para traer los indicadores financieros
sql_indicators='''
    SELECT
        Id,
        code,
        description
    FROM
        dimIndicators   
'''

In [5]:
#Query para truncar la tabla de hechos
sql_truncate = '''
 truncate table facIndicators
'''

In [6]:
# Regresa una coneccion
def get_connection():
    return pymssql.connect(server=_server, user=_user, password=_pass, database=_db)  

In [7]:
# Query para regresar el max date por indicador
def sql_maxDate(indicator):    
        sql_indicators_max='''
                select
                    max([date]) as max_date
                from
                    facIndicators
                where
                    indicatorId={}

        '''.format(indicator)
        return sql_indicators_max

In [8]:
#Funcion para regresar los indicadores financieros como un pandas dataFrame
def return_indicators():
    conn = pymssql.connect(server=_server, user=_user, password=_pass, database=_db)  
    cursor=conn.cursor()
    cursor.execute(sql_indicators)
    return pd.read_sql(sql_indicators,conn)

Get Yahoo Finance Data

In [9]:
# Funcion para traer datos 
# load_init=0 trae los datos mas recientes
# load_init=1 hacer una carga inicial con toda la información
# Regresa la información como un pandas dataFrame

def get_financial_data_code(code='BTC-USD', date_star=datetime.now(), date_end=datetime.now(),load_init=0):
      if load_init==0:
            df = web.DataReader(code, 'yahoo', start=date_star, end=date_end)  
      else:
            df = web.DataReader(code, 'yahoo')
      return df

In [10]:
#regresa el max date por inidicador financiero
def get_financial_data(indicatorId,code):
      sql=sql_maxDate(indicatorId)            
      df=pd.read_sql(sql,get_connection())
      return df    

In [11]:
#Funcionar para insertar todos los datos
def insert_financial_data(id,df):  
    connection = get_connection()
    cursor = connection.cursor()      
    for index, row in df.iterrows():
            sql_insert='''
                        insert into facIndicators
                        (                    
                            indicatorId,
                            [date],
                            hight,

                            [low],
                            [open],
                            [close],

                            volume,
                            adj_close                                              
                        )
                        values
                        (
                            {},
                            {},
                            {},

                            {},
                            {},
                            {},

                            {},
                            {}                            
                        )
            '''.format(
                        id, 
                        row["Date"].strftime("'%Y-%m-%d'"),                       
                        round(float(row["High"]),2),
                        round(float(row["Low"]),2),
                        round(float(row["Open"]),2),
                        round(float(row["Close"]),2),
                        round(float(row["Volume"]),2),
                        round(float(row["Adj Close"]),2),
                    )    
            cursor.execute(sql_insert)
            
    # commit the changes
    connection.commit()
    
    # close the connection
    connection.close()

In [11]:
# trunca la tabla para llenarla de nuevo
def truncate_financial_data():  
    connection = get_connection()
    cursor = connection.cursor()                             
    cursor.execute(sql_truncate)  
              
    # commit the changes
    connection.commit()
    # close the connection
    connection.close()



Read financial Data

In [12]:
# trae los datos de los indiadores financieros a llenar
indicator_df=return_indicators()

#limpio la tabla -- procedimiento temporal
truncate_financial_data()

#Por cada indicador trae su información y se inserta
for index, row in indicator_df.iterrows():                
        data_df=get_financial_data_code(row['code'],datetime.now(),datetime.now(),1)
        data_df.reset_index(inplace=True)
        insert_financial_data(int(row['Id']),data_df)